In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun 29 07:14:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install numpy==1.19.5

In [ ]:
import numpy
numpy.version.version

'1.19.5'

In [ ]:
import scipy
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import cv2

class DataLoader():
    def __init__(self, img_res=(256, 256)):        
        self.img_res = img_res
        
    def load_batch(self, batch_size=1, is_testing=False):        
        path = glob('/content/drive/MyDrive/Model/training_png/*')
       
        self.n_batches = int(len(path) / batch_size)

        for i in range(self.n_batches - 1):
            batch = path[i * batch_size:(i + 1) * batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
              
                img_r = cv2.imread(img)
                # Transform from bgr to rgb
                img_r = cv2.cvtColor(img_r, cv2.COLOR_BGR2RGB)
                h, w, _ = img_r.shape
                half_w = int(w / 2)
                img_A = img_r[:, :half_w, :]
                img_B = img_r[:, half_w:, :]
                # Resize to 256,256
                img_A = resize(img_A, self.img_res)
                img_B = resize(img_B, self.img_res)
                # Append to list             
                imgs_A.append(img_A/255.)
                imgs_B.append(img_B/255.)

            imgs_A = np.array(imgs_A)
            imgs_B = np.array(imgs_B)

            yield imgs_A, imgs_B


In [ ]:
from tensorflow.keras.layers import ZeroPadding2D, Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten

import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *

from tensorflow.keras.layers import PReLU
class Discriminator():
    def __init__(self,img_shape=(256,256,3)):
        self.img_shape = img_shape
    
    # Pyramidal Pooling
    def Pyramid_Pool(self, layer_input):
        x_list = [layer_input]

        def Pool():
            x = MaxPooling2D(pool_size=(2, 2))(layer_input)           
            x = self.Deconv2d_block(x, 2)
            return x

        # First level
        x_list.append(Pool())
        # Second level
        x2 = MaxPooling2D(pool_size=(4, 4))(layer_input)
        x2 = self.Deconv2d_block(x2, 2)
        x2 = self.Deconv2d_block(x2, 2)
        x2 = ZeroPadding2D(padding=(1, 1))(x2)
        x_list.append(x2)
        # Third level
        x3 = MaxPooling2D(pool_size=(8, 8))(layer_input)
        x3 = self.Deconv2d_block(x3, 4)
        x3 = self.Deconv2d_block(x3, 4)
        x3 = self.Deconv2d_block(x3, 4)
        x3 = ZeroPadding2D(padding=(3, 3))(x3)
        x_list.append(x3)

        x = Concatenate(axis=-1)(x_list)
        return x

    # Discriminator Layer
    def d_layer(self, layer_input, filters, kernel=4, bn=True):

        x = Conv2D(filters, kernel_size=kernel, strides=1)(layer_input)
        x = PReLU()(x)
        if bn:
            x = BatchNormalization(momentum=0.8)(x)
        x = MaxPooling2D()(x)
        return x

    # Deconv Layer
    def Deconv2d_block(self, layer_input, filters, kernel=4):
        x = UpSampling2D(size=2)(layer_input)
        x = Conv2D(filters, kernel_size=kernel, strides=1, padding='same', activation='relu')(x)        
        x = BatchNormalization(momentum=0.8)(x)
        return x

    

    def create_discriminator(self):
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)
        combined_imgs = Concatenate(axis=-1)([img_A, img_B])

        x0 = self.d_layer(combined_imgs, 64, 3)
        x1 = self.d_layer(x0, 256, 3)
        x2 = self.d_layer(x1, 512, 3)
        x3 = self.d_layer(x2, 64, 3)
        x4 = self.Pyramid_Pool(x3)
        # Output c is 72
        out = Activation('sigmoid')(x4)

        return Model([img_A, img_B], out)


In [ ]:
from tensorflow.keras.layers import ZeroPadding2D, Concatenate
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
import tensorflow as tf


class Generator:
    def __init__(self, img_shape = (256,256,3)):
        self.img_shape = img_shape
    
    # Dense Block with skip connection
    def dense_block(self, layer_input, num_layers):  
        x_list = [layer_input]
        for i in range(num_layers):
            x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(layer_input)
            x = BatchNormalization()(x)
            x = LeakyReLU()(x)
            
            x_list.append(x)
            x = Concatenate(axis=-1)(x_list) 
        return x    
    
    def Conv2d_block(self, layer_input, n_filters, kernel, stride, padding='valid'):  # Generator Convolution Layer  
        x = Conv2D(filters=n_filters, kernel_size=kernel, strides=stride, padding=padding)(layer_input)
        x = BatchNormalization(momentum=0.8)(x)
        x = Activation('relu')(x)
        
        return x    

    def Deconv2d_block(self, layer_input, n_filters):
        x = UpSampling2D(size=2)(layer_input)
        x = Conv2D(n_filters, kernel_size=4, strides=1, padding='same', activation='relu')(x)
        x = BatchNormalization(momentum=0.8)(x)
        return x

    def create_generator(self):
        inp = Input(shape=self.img_shape)
        
        # DownSampling 
        x0 = self.Conv2d_block(inp, 64, (3, 3), (1, 1))
        x0 = MaxPooling2D()(x0)

        x1 = self.dense_block(x0, 4)
        x1 = self.Conv2d_block(x1, 128, (3, 3), (2, 2))

        x2 = self.dense_block(x1, 6)
        x2 = self.Conv2d_block(x2, 256, (3, 3), (2, 2))

        # No sampling
        x3 = self.dense_block(x2, 8)
        x3 = self.Conv2d_block(x3, 512, (3, 3), (1, 1), padding='same')

        x4 = self.dense_block(x3, 8)
        x4 = self.Conv2d_block(x4, 512, (3, 3), (1, 1), padding='same')

        x5 = self.dense_block(x4, 8)
        x5 = self.Conv2d_block(x5, 128, (3, 3), (1, 1), padding='same')

      # UpSampling 
        x6 = self.dense_block(x5, 6)
        x6 = self.Deconv2d_block(x6, 120)

        x7 = self.dense_block(x6, 4)
        x7 = self.Deconv2d_block(x7, 64)

        x8 = self.dense_block(x7, 4)
        x8 = self.Deconv2d_block(x8, 64)

        x9 = self.dense_block(x8, 4)
        x9 = self.Conv2d_block(x9, 16, (3, 3), (1, 1), padding='same')

        x10 = ZeroPadding2D(padding=(5, 5))(x9)

        x11 = Conv2D(filters=3, kernel_size=(3, 3))(x10)
        out = Activation('tanh')(x11)

        return Model(inp, out)


In [ ]:
from __future__ import print_function, division
import scipy
import cv2
import pandas
import tensorflow.keras.backend as K
from tensorflow.keras.layers import ZeroPadding2D, Concatenate
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Conv2DTranspose
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam,SGD
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import model_from_json
import datetime
import matplotlib.pyplot as plt
import sys
from tensorflow.keras.layers import PReLU
import numpy as np
import os
import scipy.misc
from glob import glob
import tensorflow as tf


class GAN():
    def __init__(self):
        self.img_shape = (256,256,3)
        
        
        # Instance of data loader
        self.data_loader = DataLoader(img_res=(256,256))
        
        self.disc_out = (14, 14, 72)  

        self.generator = Generator().create_generator()
        self.discriminator = Discriminator().create_discriminator()
        self.GAN_model = self.build_GAN() 
         
         
        self.gan_optimizer = Adam(learning_rate=1E-3, beta_1=0.3, beta_2=0.99, epsilon=1e-08,amsgrad=True)         
        self.discriminator_optimizer = Adam(learning_rate=1E-3, beta_1=0.5, beta_2=0.99, epsilon=1e-08,amsgrad=True)

    def build_GAN(self):
        self.discriminator.trainable = False
        
        img_B = Input(shape=self.img_shape)       
        fake_A = self.generator(img_B)
        
        discriminator_output = self.discriminator([fake_A, img_B])
        
        GAN_model = Model(inputs=[img_B],
                           outputs=[fake_A, fake_A, discriminator_output],
                           name='CGAN')  
        return GAN_model

    def train(self, epochs, batch_size=10, sample_interval=80):              
        def perceptual_loss(img_true, img_generated):  
            image_shape = self.img_shape
            vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_shape)
            
            loss_block3 = Model(vgg.input, vgg.get_layer('block3_conv3').output)
            loss_block3.trainable = False
            loss_block2 = Model(vgg.input, vgg.get_layer('block2_conv2').output)
            loss_block2.trainable = False
            loss_block1 = Model(vgg.input, vgg.get_layer('block1_conv2').output)
            loss_block1.trainable = False
            return np.mean(K.square(loss_block1(img_true) - loss_block1(img_generated))) + K.mean(
                K.square(loss_block2(img_true) - loss_block2(img_generated))) + K.mean(
                K.square(loss_block3(img_true) - loss_block3(img_generated)))
        
        # Set the Discriminator to false for training Generator  
        self.discriminator.trainable = False  
        # Compile the Generator
        self.generator.compile(loss=perceptual_loss, optimizer=self.gan_optimizer,run_eagerly=True)  
        # Three Loses
        CGAN_loss = ['mae', perceptual_loss, 'mse']  
        CGAN_loss_weights = [6.6e-3, 1, 6.6e-3]
        self.GAN_model.compile(loss=CGAN_loss, loss_weights=CGAN_loss_weights,
                                optimizer=self.gan_optimizer,run_eagerly=True)
        
        # To train the Discriminator set trainable to true
        
        self.discriminator.trainable = True
        self.discriminator.compile(loss="mse",
                                   optimizer=self.discriminator_optimizer)
        start_time = datetime.datetime.now()
        # Real world Images
        
        valid = np.ones((batch_size,) + self.disc_out) 
        # Generated Images
        fake = np.zeros((batch_size,) + self.disc_out)  

        for epoch in range(epochs):
            if epoch == 0:
              try: 
                self.generator.load_weights("/content/drive/MyDrive/Model/gen_model_v2.h5")
                self.discriminator.load_weights('/content/drive/MyDrive/Model/dis_model_v2.h5')
                self.GAN_model.load_weights('/content/drive/MyDrive/Model/com_model_v2.h5')
              except:
                pass
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):
                # Generated Image
                fake_A = self.generator.predict(imgs_B)  
                
                d_loss_real = self.discriminator.train_on_batch([imgs_A, imgs_B], valid)
                d_loss_fake = self.discriminator.train_on_batch([fake_A, imgs_B], fake)
                # Discriminator Loss
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)  
                # Train the Combined model
                self.GAN_model.trainable = True  
                self.discriminator.trainable = False
                g_loss = self.GAN_model.train_on_batch(imgs_B, [imgs_A, imgs_A, valid])

                elapsed_time = datetime.datetime.now() - start_time
                if batch_i % sample_interval == 0:
                  print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] time: %s" % (epoch, epochs,
                                                                                          batch_i,
                                                                                          self.data_loader.n_batches,
                                                                                          d_loss,
                                                                                          g_loss[0],
                                                                                          elapsed_time))

                  
                  imgshow = np.zeros((256,512,3))
                  imgshow[:,:256,:] = (imgs_A[0,:,:,:]  * 255.)
                  imgshow[:,256:,:] = (fake_A[0,:,:,:]  * 255.)
                  plt.imshow(imgshow)
                  plt.show()

            # Save weights after each epoch
            self.GAN_model.save_weights("/content/drive/MyDrive/Model/com_model_v2.h5")
            self.generator.save_weights("/content/drive/MyDrive/Model/gen_model_v2.h5")
            self.discriminator.save_weights("/content/drive/MyDrive/Model/dis_model_v2.h5")


In [ ]:
with tf.device('/GPU:0'):
  gan = GAN()
  gan.train(epochs=500, batch_size=10, sample_interval=80)

In [ ]:
gan = GAN()
gan.get_weights()

AttributeError: ignored

In [ ]:
from math import log10, sqrt
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [ ]:
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse
from google.colab.patches import cv2_imshow
psnrs = []
ssims = []
path = glob("/content/drive/MyDrive/Model/testing/*")
gan=GAN()
test_model = gan.generator
test_model.load_weights("/content/drive/MyDrive/Model/gen_model_v2.h5")
for i in range(len(path)):
  img_B = plt.imread(path[i])  
  try:
    h, w, d = img_B.shape
    half_w = int(w/2)
    img_show = np.zeros((h,2*w, d))
  except:
    continue
  img_b = img_B[:, half_w:, :] 
  ground_t = img_B[:, :half_w, :] 

  ground_t = cv2.resize(ground_t, (256,256)) 
  img_b = cv2.resize(img_b, (256,256)) 

  # plt.imshow(ground_t)
  # plt.show()
  img_b = np.expand_dims(img_b, axis=0)
  img_b_norm = np.array(img_b) / 255.

  fake = test_model.predict(img_b_norm)* 255.

  score = ssim(fake[0,:,:,:], ground_t, multichannel=True)
  # ssims.append(score)
  # psnrs.append(PSNR(fake,ground_t))
  img_show = np.zeros((256,3*256,3))
  img_show[:,:256,:] = img_b[0,:,:,:]
  img_show[:,256:512,:] = fake[0,:,:,:]
  img_show[:,512:,:] = ground_t

  # plt.imshow((fake[0,:,:,:]))
  cv2.imwrite(f'/content/drive/MyDrive/Model/results/{i}.png',255*img_show)
  print(img_show.shape)
  plt.imshow(img_show)
  plt.show()

  print("SSIM - ",score)
  print("PSNR - ",PSNR(fake,ground_t))

In [ ]:
  print("Mean SSIM - ", np.mean(np.array(ssims)))
  print("Mean PSNR - ", np.mean(np.array(psnrs)))

Mean SSIM -  0.6064853547339741
Mean PSNR -  61.5259045045339
